In [29]:
## ---- Partition Cell Set Correlation Analysis-Network Notebooks C ---- 0650.00.00
## Load Analysis Parameters (Parm1)
## Loop Through Each Partition and Perform Analysis as Follows:
## Load Partition Cell Set
## Load Partition Differential Expresssion Coefficients
## Perform Correlation Analysis
## Perform Perform FDR Scoring
## Create Network with Extreme Cut
## Annotate Network with Differential Expression Results
## Plot Network as Needed
##
## Note that this notebook was originally coded for partial correlation. It was recoded here for correlation
## the used of variable names and comments referring the partial correlation should be read as
## correlation. Output file names have been amended accordingly, and the FDR cutoff parameter is differeent
## from that used in partial correlation network analysis. The level is different, and pvalue is used instead

In [1]:
## Create a Working Input and Output Data Directory, If Id Does Not Exist
parentdir <- '/gpfs/group/torkamani/devans/'
datdir <- paste(parentdir, 'CDC2', sep = '')
if (!file.exists(datdir)) {
    dir.create(datdir)
}
setwd(datdir)

In [31]:
## Read the parameters file
ps <- read.table(file = 'parms.txt', stringsAsFactors = FALSE, header = TRUE)

In [32]:
## Read the GENCODE v27 Data
v27_gen <- read.table(paste(ps$indir, 'v27_Embl_Hugo.txt', sep = ''), header = F)
v27_gen[,1] <- substr(v27_gen[,1] ,1, 15)
v27 <- read.table(paste(ps$indir, 'v27_Embl_Hugo_Type.txt', sep =''))

In [33]:
## Load Monocle3 and Seurat Libraries
library(monocle3)
library(Seurat)
library(dplyr)
library(magrittr)
library(ggplot2)
library(gridExtra)
library(Matrix)
library(rhdf5)
library(grid)
library(igraph)
library(corpcor)
library(fdrtool)
## Get the igraph utilities
source('/gpfs/home/sjsmith/code/xtissue_paper/notebooks/Utilities/setup_igraph.r')
## Alternate to stat_parm utility in igraph utilities (does not assume log10 data)
stat_parm2 <- function(gf, expdata){
    
    ## Remove expression values not in the network, get antilog
    # expdata_filt <- 10 ^ log10expdata[,V(gf)$name]
    expdata_filt <- expdata[,V(gf)$name]
    
    ## Compute the node expression stats, including coeff of variation, add to network
    V(gf)$mean <- apply(expdata_filt, 2, mean)
    V(gf)$sd <- apply(expdata_filt, 2, sd)
    V(gf)$cv <- V(gf)$sd/V(gf)$mean
    V(gf)$med <- apply(expdata_filt, 2, median)

    ## Compute the strength
    V(gf)$strength <- strength(gf, weights = abs(E(gf)$pcor))
    return(gf)
    }
## Alternate to plot function in utility

plot_parm2 <- function (gf) {

    V(gf)$color <- 'red'
    
    E(gf)$color[E(gf)$pcor > 0] <- 'green'
    E(gf)$color[E(gf)$pcor < 0] <- 'orange'
    fedthick <- 100 * abs(E(gf)$pcor)
    V(gf)$label.cex = .1
    E(gf)$label.cex = .1
    V(gf)$label <- paste(substr(V(gf)$v27,1,10),
                                   round(V(gf)$strength,3), sep = '\n')
    E(gf)$width <- fedthick
    E(gf)$label <- round(E(gf)$pcor, 4)
    V(gf)$size <- 1

    return(gf)
    }

In [34]:
## Read the previously preprocessed downsampled cell set data object
down_stdycds <- readRDS(file = paste(ps$outdir,
            'Aggregated.downsampled.QC.NoDoublets.Repartitioned.rds', sep = ''))

In [35]:
## Build a gene short name to gene id (Ensembl) lookup
short2geneid <- fData(down_stdycds)@rownames
names(short2geneid) <- fData(down_stdycds)@listData$gene_short_name

In [36]:
## Build a gene id (Ensembl) to gene short name lookup
geneid2short <- fData(down_stdycds)@listData$gene_short_name
names(geneid2short) <- fData(down_stdycds)@rownames

In [37]:
## Create variables for how cells sets are organized
cellgrps <- c('healthy', 'diseased', 'healthy', 'diseased', 'healthy', 'diseased')
cellpats <- c('ID Number 1', 'ID Number 1', 'ID Number 2', 'ID Number 2', 'ID Number 3', 'ID Number 3')

In [38]:
## Define and Assign Cell Types
celltypes6 <- c('1-Macrophages',
                '2-Endothelial Cells',
                '3-VSMCs',
                '4-Natural Killer Cells',
                '5-Cytotoxic T Lymphocytes',
                '6-B Lymphocytes')

In [39]:
## Declare Tom's best genes for definiting cell types
toms_markers5 <- c('NRXN1', 'CLU', 'ICAM2',
                 'CD14', 'CD68', 'AIF1',
                 'VWF', 'EDN1', 'ECSCR',
                 'MKI67', 'UBE2C', 'TOP2A',
                 'ACTA2', 'TAGLN', 'MYL9',
                 'ACKR1', 'SPARCL1', 'PECAM1',
                 'CALD1', 'MGP', 'DCN',
                 'NKG7', 'XCL1', 'CTSW',
                 'CD8A', 'TRAC', 'CD2',
                 'MS4A1', 'CD79A', 'BANK1',
                 'CD69', 'CXCR4', 'IL7R',
                 'LILRA4', 'IRF7', 'CLEC4C',
                 'MZB1', 'JCHAIN', 'TNFRSF17',
                 'LST1', 'FCGR3B', 'S100A8',
                 'TPSAB1', 'CPA3', 'MS4A2')
toms_gene_ids5 <- short2geneid[toms_markers5]


doug_markers1 <- c('AIF1', 'LYZ', 'FCER1G',  'CD68',
                'RNASE1', 'PECAM1', 'IGFBP4', 'ADIRF', 
                'SOD3', 'MYL9', 'CALD1', 'GSN',
                'TYROBP', 'NKG7', 'CTSW', 'CD69',
                'CD3D', 'CD2', 'TRBC2', 'TRAC',
                'MS4A1', 'CD79A', 'HLA-DQA1', 'CD37')
dougs_gene_ids1 <- short2geneid[doug_markers1]

In [43]:
## Loop through the partitions and perform analysis, making networks
## Determine the number of partitions
np <- length(celltypes6)

## Some control parameters
gout <- TRUE
gene_cut <- .05              # Average fraction of UMI's per cell needed to keep a gene (1/100 typical)
max_fdr_cut <- 20000         # The maximum number of significant edges to save
graph_edges <- 500           # The number of edges to plot
cutoff <- 1E-1               # FDR p-value cutoff for correlations
diff_exp_qval_cut <- 0.05    # Differential Expression q-value cutoff
min_sig_fdr <- 10            # There needs to be at least 10 significant edges to form a network

for (p in 1:np) {
    setwd(paste(datdir, '/', ps$outdir, celltypes6[p], sep = ''))
    ## Extract partition subset for par
    partn_cds <- readRDS('Partition.Cell.Set.rds')
    if (file.exists('Diff.AllHealthCoeff.RemoveFail.NoModels.txt'))  {   
        de_tab <- read.table('Diff.AllHealthCoeff.RemoveFail.NoModels.txt', header = TRUE,
                             stringsAsFactors = FALSE)
    }
    partn_exp <- exprs(partn_cds)
    ## There are lots of low expressing genes
    ## Let's remove them (at least 1 UMI per 100 cells). Transpose the results
    sig_genes <- (rowSums(partn_exp) > dim(partn_exp)[2] * gene_cut)
    partn_exp <- partn_exp[sig_genes, ]
    ## There may also be some cells with no variance. Remove those too
    sig_cells <- (apply(partn_exp, 2, var) > 0)
    partn_exp <- partn_exp[, sig_cells]
    ## Transpose the results for next steps
    partn_exp <- t(partn_exp)
#     print(sum(sig_genes))
#     print(sum(sig_cells))
#     print(dim(partn_exp))
    ## Compute the partial correlations
    pc_partn_exp <- cor.shrink(partn_exp)   
    ## Convert them to a vector
    pcv_partn <- sm2vec(pc_partn_exp)
    pcv_partn_ind <- sm.index(pc_partn_exp)
#     print(length(pcv_partn))
    ## Compute the FDR scores
    pdf('FDR.Tool.Results.cor.pdf', width = 8, height = 6)
        pcv_partn_fdr <- fdrtool(pcv_partn, statistic = "correlation")
    dev.off()
    ## Gete the expression data gene names
    pcor_v_names <- colnames(partn_exp)
    # Create an ordered set of the correlation data and fdrtools data. Include columns
    # with pointers to the partial correlation matrix row and column so that the original
    # data can referenced.
    pcor_order <- order(pcv_partn, decreasing = FALSE)
    composite <- cbind(pcv_partn, pcv_partn_fdr$pval, pcv_partn_fdr$qval, pcv_partn_fdr$lfdr,
               pcor_order, pcv_partn_ind[,1], pcv_partn_ind[,2])
    # Sort the array by partial correlation values
    com_s <- composite[pcor_order,]
    rcount <- length(com_s[,1])
    # Add a column with the row number (after sort)
    com_s <- cbind(com_s, 1:rcount)
    # Make nicer column names
    colnames(com_s) <- c('pcor', 'pval', 'qval', 'lfdr', 'ord', 'row', 'col', 'idx')
    # Get the id of the last value in fdr, then create cut set
    cutset <- com_s[,2] <= cutoff # changed to use p-value
    fdr_cut <- com_s[cutset,]
    fdr_cut <- as.data.frame(fdr_cut)
    ## Break out if fdr_cut is empty
    if (dim(fdr_cut)[1] <= min_sig_fdr) break
    ## Cut down the number of edges is greater than max_fdr_cut (typically only save 20,000)
    if (dim(fdr_cut)[1] > max_fdr_cut) {
        qorder <- order(fdr_cut$qval, decreasing = FALSE)
        fdr_cut <- fdr_cut[qorder[1:max_fdr_cut],]
        pcorder <- order(fdr_cut$pcor, decreasing = FALSE)
        fdr_cut <- fdr_cut[pcorder,]
    }
    # Add a column in the fdr data for the TO node (for edge defintion)
    fdr_cut$to <- pcor_v_names[fdr_cut$row]
    # Add a column in the fdr data for the FROM node (for edge defintion)
    fdr_cut$from = pcor_v_names[fdr_cut$col]
    # Compute a vector containing the ordering of the absolute value of the partial correlation
    arank_v <- order(abs(fdr_cut$pcor), decreasing = T)
    # Store this rank in the fdr data
    fdr_cut$arank[arank_v] <- 1:length(fdr_cut[,1])
    # Add the gene attributes to a node data frame and set the column names
    nodes <- pcor_v_names
    # Convert the fdr data and node data frame into a graph
    # net <- graph_from_data_frame(d=fdr_cut[,c(9:10,1:8,11)], vertices=nodes, directed=F)
    net <- graph_from_data_frame(d=fdr_cut[,c('to', 'from', 'pcor', 'pval', 'qval', 'lfdr',
                                         'ord', 'row', 'col', 'idx', 'arank')],
                                         vertices=nodes, directed=F)
    # Show some parameters of the fdr data and the graph (number of nodes - last line)
    ## Write the network object and the FDR object
    saveRDS(net, 'Network.cor.rds')
    saveRDS(fdr_cut, "FDR_CUT.cor.rds")
    tonames <- unique(fdr_cut[,9])
    fromnames <-  unique(fdr_cut[,10])
    allnames <- nodes
    # Get all the gene names of nodes
    cnode_names <- union(tonames, fromnames)
    # Create a logical vector for connected node names
    bvec <- is.element(V(net)$name, cnode_names)
    # Recreate the graph, but don't include unconnected nodes
    net2 <- delete_vertices(net, nodes[!bvec])
    # Set the v27attribute equal to the looked up gene name
    V(net2)$v27 <- as.vector(v27_gen[match(V(net2)$name, v27_gen[,1]),2])
    # Lookup the HUGO gene names and types for the network nodes
    hv27types <- as.vector(v27[match(V(net2)$name, substr(v27[,1],1,15)),3])
    # Save the gene types for V27 as attributes in the network
    V(net2)$v27type <- hv27types
    ## Set the weight equal to the recipicol of the absolute pcor values
    ## This is an interpretation of weight = distance
    E(net2)$weight <- 1/abs(E(net2)$pcor)
    # Given node 1 is highly connected,
    # find unreachable nodes from it
    a <- dfs(net2, 1, neimode = 'all', unreachable = FALSE)
    # Delete all but the unreach nodes from the main subnet and check the change
    net2 <- delete_vertices(net2, V(net2)[-a$order[!is.na(a$order)]])
    ## add degree and stength node attributes
    V(net2)$strength <- strength(net2, weights = abs(E(net2)$pcor))
    V(net2)$degree <- degree(net2, normalized = TRUE)
    # Get just the top edges of the graph (fixed to be ordered by pcor)
    net2_edge_o <- order(abs(E(net2)$pcor), decreasing = TRUE)
    ## Remove the small graph, it's not needed, and fixed size is sometimes a problem 
    # gs1 <- subgraph.edges(net2, E(net2)[net2_edge_o[1:250]], delete.vertices = TRUE)
    ## Set up for a larger graph later (1000 edges for the number of edges in the network, whichever is less)
    gs1.2 <- subgraph.edges(net2,
        E(net2)[net2_edge_o[1:min(graph_edges, length(E(net2)))]], delete.vertices = TRUE)         
    ## Get the mean, sd, and coeff of variation for all the genes, set up for annotation
    gs2 <- stat_parm2(gs1.2, partn_exp)
    V(gs2)$gene <- V(gs2)$name
    ## Get the annotation information, force single tissue (cell type)
    gs3 <- annot_parm(gs2)
    ## Get the plot information
    gs4 <- plot_parm2(gs3)
    ## Create temporary clusters to compute a component layout
    ## This is the list of plot algorithms
    lmx <- c(component_wise, layout_as_bipartite, layout_as_star, layout_as_tree,
         layout_in_circle, layout_nicely, layout_on_grid, layout_on_sphere, layout_randomly,
         layout_with_dh, layout_with_fr, layout_with_gem, layout_with_graphopt, layout_with_kk,
         layout_with_lgl, layout_with_mds, layout_with_sugiyama, merge_coords, norm_coords, normalize)

    lt <- layout_with_dh # layout_nicely # layout_with_graphopt
    # Create a temporary clustering object
    # And break cross module links for component layout
    clust_temp <- cluster_louvain(gs4)
    sum(crossing(clust_temp, gs4))
    gs4_temp <- delete.edges(gs4, E(gs4)[crossing(clust_temp, gs4)])
    set.seed(103)
    gs4_lo <- layout_components(gs4_temp, layout = lt)
    gs4_lo <- layout.norm(gs4_lo, -1, 1, -1, 1)
    ## Get the significant differential expressed terms
    qsig <- de_tab %>% filter(q_value < diff_exp_qval_cut) %>% 
        dplyr::select(gene_short_name, normalized_effect, q_value)
    ## If there are any significant DE genes, colorize them in the network
    V(gs4)$color <- rgb(0, 0, 1, alpha = .75)
    valid_de <- dim(qsig)[1]
    if (valid_de >= 1) {
        gene2ne <- qsig$normalized_effect
        names(gene2ne) <- qsig$gene_short_name
        gene2ne <- gene2ne/max(abs(gene2ne))
        gene2ne_p <- round(gene2ne[gene2ne > 0], 3)
        gene2ne_m <- round(-gene2ne[gene2ne < 0], 3)
        V(gs4)$color <- rgb(0, 0, 1, alpha = .75)
        V(gs4)$color[V(gs4)$v27 %in% names(gene2ne_p)] <- 
            rgb(1, gene2ne_p[names(gene2ne_p) %in% V(gs4)$v27], 
                gene2ne_p[names(gene2ne_p) %in% V(gs4)$v27], alpha = .5)
        V(gs4)$color[V(gs4)$v27 %in% names(gene2ne_m)] <- 
            rgb(gene2ne_m[names(gene2ne_m) %in% V(gs4)$v27], 1,
                gene2ne_m[names(gene2ne_m) %in% V(gs4)$v27], alpha = .5)   
    }
    ## Set up more plotting parameters
    ## Node boarder color 
    V(gs4)$v27type[which(is.na(V(gs4)$v27type))] <- 'unknown'  ## Fix items with not biotype
    pcnodes <- which(V(gs4)$v27type == 'protein_coding')
    npcnodes <- which(V(gs4)$v27type != 'protein_coding')
    V(gs4_temp)$color <- V(gs4)$color
    V(gs4_temp)$v27type <- V(gs4)$v27type
    pccol <- rgb(1, 0, 0, alpha = .75)
    npccol <- rgb(0, 1, 0, alpha = .75)
    node_frames_cols <- vector(mode = "character", length = length(pcnodes))
    node_frames_cols[pcnodes] <- pccol
    node_frames_cols[npcnodes] <- npccol
    ## Module label color (translucent blue)
    nodelabc <- rgb(0, 0, 1, alpha = .4)
    # Module background color (make it clear)
    polycol <- rgb(0, 1, 1, alpha = .0)    
    ## Set up for plotting of cluster ids (x and y coordinates)
    numcl <- length(clust_temp)
    ctxtpos <- matrix(0, nrow = numcl, ncol = 2)
    colnames(ctxtpos) <- c('x', 'y')
    for (i in 1:numcl) {
        ctxtpos[i,'x'] <- max(gs4_lo[membership(clust_temp) == i, 1])
        ctxtpos[i,'y'] <- mean(gs4_lo[membership(clust_temp) == i, 2])
        }  

    ## set up plot file
    plotfp <- 'PartitionNetworkPlot.1000.DiffExp.wCrossings.cor.Rev2.pdf'     
    pdf(plotfp, width = 30, height = 30)
    plot(gs4, layout = gs4_lo, mark.groups = clust_temp, mark.shape = 1,
        mark.expand = 1.5, mark.col = polycol,
        vertex.frame.color = node_frames_cols, vertex.shape = 'circle',
        edge.width = 2 * abs(E(gs4)$pcor), vertex.size = .6, 
        main = "Correlation Network With Crossings")
    
    legend(x = -0.85, y = -.85, legend = c('Protein Coding biotype genes have Red Frames',
                                        'All Other biotype genes have Green Frames',
                                        'Nodes Shaded Blue Are Not Effected By Disease State',
                                        'Red Is Healthy, Green Is Diseased',
                                        'Strength Shown Below Gene Name in Each Node'), col = 'black')
    text(ctxtpos[,1], ctxtpos[,2], adj = c(-.25, .5), 1:numcl, cex = 1, col = nodelabc, font = 4)
    dev.off()
    
    ## set up plot file
    plotfp <- 'PartitionNetworkPlot.1000.DiffExp.woCrossings.cor.Rev2.pdf'     
    pdf(plotfp, width = 30, height = 30)
    plot(gs4_temp, layout = gs4_lo, mark.groups = clust_temp, mark.shape = 1,
        mark.expand = 1.5, mark.col = polycol,
        vertex.frame.color = node_frames_cols, vertex.shape = 'circle',
        edge.width = 2 * abs(E(gs4)$pcor), vertex.size = .6, 
        main = "Correlation Network Without Crossings")
    
    legend(x = -0.85, y = -.85, legend = c('Protein Coding biotype genes have Red Frames',
                                        'All Other biotype genes have Green Frames',
                                        'Nodes Shaded Blue Are Not Effected By Disease State',
                                        'Red Is Healthy, Green Is Diseased',
                                        'Strength Shown Below Gene Name in Each Node'), col = 'black')
    text(ctxtpos[,1], ctxtpos[,2], adj = c(-.25, .5), 1:numcl, cex = 1, col = nodelabc, font = 4)
    dev.off()  
}

Estimating optimal shrinkage intensity lambda (correlation matrix): 0.0711 

Step 1... determine cutoff point
Step 2... estimate parameters of null distribution and eta0
Step 3... compute p-values and estimate empirical PDF/CDF
Step 4... compute q-values and local fdr
Step 5... prepare for plotting

Estimating optimal shrinkage intensity lambda (correlation matrix): 0.137 

Step 1... determine cutoff point
Step 2... estimate parameters of null distribution and eta0
Step 3... compute p-values and estimate empirical PDF/CDF
Step 4... compute q-values and local fdr
Step 5... prepare for plotting

Estimating optimal shrinkage intensity lambda (correlation matrix): 0.0634 

Step 1... determine cutoff point
Step 2... estimate parameters of null distribution and eta0
Step 3... compute p-values and estimate empirical PDF/CDF
Step 4... compute q-values and local fdr
Step 5... prepare for plotting

Estimating optimal shrinkage intensity lambda (correlation matrix): 0.7045 

Step 1... determine c

In [44]:
np
p

[1] 6

[1] 6